# Fire Agent with LlamaIndex

## Install Dependencies

In [1]:
# !pip install uv
# !uv pip install --system -qU llama-index==0.11.6 llama-index-llms-openai llama-index-readers-file llama-index-embeddings-openai llama-index-llms-openai-like "openinference-instrumentation-llama-index>=2" arize-phoenix python-dotenv

  Using cached uv-0.4.27-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
Using cached uv-0.4.27-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.8 MB)


## Setup API Keys
To run the rest of the notebook you will need access to an OctoAI API key. You can sign up for an account [here](https://octoai.cloud/). If you need further guidance you can check OctoAI's [documentation page](https://octo.ai/docs/getting-started/how-to-create-octoai-access-token).

In [1]:
from os import environ
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = environ["OPENAI_API_KEY"]

## Import libraries and setup LlamaIndex

In [2]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI


# Create an llm object to use for the QueryEngine and the ReActAgent
llm = OpenAI(model="gpt-4")

# Set up Phoenix

In [3]:
import phoenix as px
session = px.launch_app()

/home/hongeinh/anaconda3/envs/hackathon/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [4]:
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor
from phoenix.otel import register

tracer_provider = register()
LlamaIndexInstrumentor().instrument(tracer_provider=tracer_provider)

🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: default
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



## Load Documents

In [5]:
try:
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/nfpa"
    )
    nfpa_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False

This is the point we create our vector indexes, by calculating the embedding vectors for each of the chunks. You only need to run this once.

In [6]:
if not index_loaded:
    # load data
    nfpa_docs = SimpleDirectoryReader(
        input_files=["./NFPA10-2022.pdf"]
    ).load_data()

    # build index
    nfpa_index = VectorStoreIndex.from_documents(nfpa_docs, show_progress=True)

    # persist index
    nfpa_index.storage_context.persist(persist_dir="./storage/nfpa")

Generating embeddings: 100%|██████████| 145/145 [00:03<00:00, 48.05it/s]


Now create the query engines.

In [7]:
nfpa_engine = nfpa_index.as_query_engine(similarity_top_k=3, llm=llm)

We can now define the query engines as tools that will be used by the agent.

As there is a query engine per document we need to also define one tool for each of them.

In [8]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=nfpa_engine,
        metadata=ToolMetadata(
            name="NFPA",
            description=(
                "Provides information about Fire regulations for year 2022. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    )
]

## Creating the Agent
Now we have all the elements to create a LlamaIndex ReactAgent

In [9]:
agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
    max_turns=10,
)

Now we can interact with the agent and ask a question.

In [11]:
response = agent.chat("What does an ABC fire extinguisher stand for?")
print(str(response))

> Running step ca2aceb4-c7e5-4dfb-a040-7f324d34592c. Step input: What does an ABC fire extinguisher stand for?
Thought: The current language of the user is English. I need to use a tool to help me answer the question.
Action: NFPA
Action Input: {'input': 'What does an ABC fire extinguisher stand for?'}
Observation: The context does not provide information on what an ABC fire extinguisher stands for.
> Running step d5646619-ee22-4a29-95ed-ec3f4f17a3ac. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: An ABC fire extinguisher is a multipurpose extinguisher capable of putting out different types of fires. The 'A' stands for 'ordinary combustibles' such as wood or paper, 'B' stands for 'flammable liquids' such as oil or gasoline, and 'C' stands for 'electrical equipment' like appliances, wiring, circuit breakers, and outlets.
An ABC fire extinguisher is a multipurpose extinguisher capable of putting out different types of 

In [18]:
!pip install dill


I0000 00:00:1729967644.958448   36077 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers


In [20]:
import dill

with open('react_agent.dill', 'wb') as f:
    dill.dump(agent, f)
